## Transorming constructors data and storing it in silver layer for future transformations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
# importing all the required functions
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Defining the Schema for the circuits csv to read the file 

In [0]:
constructors_struct_schema = StructType([
                                         StructField("constructorId", LongType(), True),
                                         StructField("constructorRef", StringType(), True),
                                         StructField("name", StringType(), True),
                                         StructField("nationality", StringType(), True),
                                         StructField("url", StringType(), True)



])

In [0]:
#reading the constructors data from raw folder
constructors_df =  spark.read.format('json').option('header', True).schema(constructors_struct_schema).load(f"{raw_folder_path}/constructors.json")

In [0]:
#adding and dropping unnecessary columns
constructors_final_df = constructors_df.drop(col("url")).withColumnsRenamed({"constructorId" : "constructor_id", "constructorRef" : "constructor_ref"}).withColumn("ingestion_date", current_timestamp())

In [0]:
#writing the final df to silver layer
constructors_final_df.write.format('parquet').mode('overwrite').option('path',f"{silver_folder_path}/constructors").save()

In [0]:
# proofreading the loaded data
spark.read.format('parquet').option('header', True).option('inferSchema', True).load(f"{silver_folder_path}/constructors").display()